In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Large_B-cell_Lymphoma/GSE173263'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profile in DLBCL cases according to response to R-CHOP"
!Series_summary	"Gene expression profile (GEP) was analyzed in DLBCL cases to compare early failure patients (10) vs. responding patients (29), to identify features associated to primary chemoresistance"
!Series_overall_design	"We used frozen samples of DLBCL cases to analyze GEP in 2 groups: early failure (EF) cases (10), defined as refractory to induction treatment or early relapsing  (<12 months from diagnosis); and responding patients (29). Data was then analyzed with GSEA program"
Sample Characteristics Dictionary:
{0: ['tissue: Frozen lymph node biopsy'], 1: ['disease state: Diffuse large B-cell lymphoma (DLBCL)'], 2: ['response to r-chop: remission', 'response to r-chop: Early failure']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
is_gene_available = True

# Variable Availability and Data Type Conversion
# Analyze sample characteristics dictionary
sample_characteristics_dict = {
    0: ['tissue: Frozen lymph node biopsy'],
    1: ['disease state: Diffuse large B-cell lymphoma (DLBCL)'],
    2: ['response to r-chop: remission', 'response to r-chop: Early failure']
}

# Defining rows for 'Large_B-cell_Lymphoma', 'age', and 'gender'
trait_row = 2  # From 'response to r-chop'
age_row = None  # Age information is not available in the sample characteristics dictionary
gender_row = None  # Gender information is not available in the sample characteristics dictionary

# Define conversion functions
def convert_trait(value):
    if 'remission' in value:
        return 0
    if 'Early failure' in value:
        return 1
    return None

# No appropriate age and gender conversion functions are needed as the data is not available
convert_age = lambda value: None
convert_gender = lambda value: None

# Save Metadata
save_cohort_info('GSE173263', './preprocessed/Large_B-cell_Lymphoma/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Large_B-cell_Lymphoma', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Large_B-cell_Lymphoma/trait_data/GSE173263.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM5264464': [0], 'GSM5264465': [0], 'GSM5264466': [0], 'GSM5264467': [0], 'GSM5264468': [0], 'GSM5264469': [0], 'GSM5264470': [0], 'GSM5264471': [0], 'GSM5264472': [0], 'GSM5264473': [0], 'GSM5264474': [0], 'GSM5264475': [0], 'GSM5264476': [0], 'GSM5264477': [0], 'GSM5264478': [0], 'GSM5264479': [0], 'GSM5264480': [0], 'GSM5264481': [0], 'GSM5264482': [0], 'GSM5264483': [0], 'GSM5264484': [0], 'GSM5264485': [0], 'GSM5264486': [0], 'GSM5264487': [0], 'GSM5264488': [0], 'GSM5264489': [0], 'GSM5264490': [0], 'GSM5264491': [0], 'GSM5264492': [0], 'GSM5264493': [1], 'GSM5264494': [1], 'GSM5264495': [1], 'GSM5264496': [1], 'GSM5264497': [1], 'GSM5264498': [1], 'GSM5264499': [1], 'GSM5264500': [1], 'GSM5264501': [1], 'GSM5264502': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['20-Aug-10', '20-Aug-10', '20-Aug-10', '20-Aug-10', '20-Aug-10'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Description': ['g21264570 /TID=g21264570 /CNT=1 /FEA=FLmRNA /TIER=FL /STK=0 /DEF=g21264

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide the identifier and gene symbol keys
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and get the resulting gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Large_B-cell_Lymphoma/gene_data/GSE173263.csv'
os.makedirs(os.path.dirname(gene_csv_path), exist_ok=True)
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Large_B-cell_Lymphoma')

# 4. Save the cohort information.
save_cohort_info('GSE173263', './preprocessed/Large_B-cell_Lymphoma/cohort_info.json', True, True, trait_biased, merged_data)

# If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Large_B-cell_Lymphoma/GSE173263.csv'
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Large_B-cell_Lymphoma', the least common label is '1.0' with 10 occurrences. This represents 25.64% of the dataset.
The distribution of the feature 'Large_B-cell_Lymphoma' in this dataset is fine.

